# MFCC, Delta MFCC, and Chroma Lists


In [1]:
import librosa
import numpy as np
import pandas as pd

#@markdown ### Path to your audio file
audio_path = '/content/WingsofSorrow.mp3' #@param {type:"string"}
y, sr = librosa.load(audio_path, sr=None)

# Calculate MFCC, Delta MFCC, and Chroma features
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
mfcc_delta = librosa.feature.delta(mfcc)
chroma = librosa.feature.chroma_cqt(y=y, sr=sr)

# Determine the total number of frames at 10 frames per second
duration_sec = librosa.get_duration(y=y, sr=sr)
total_frames = int(duration_sec * 10)

# Resample each feature individually
features = {'MFCC': mfcc, 'MFCC_delta': mfcc_delta, 'Chroma': chroma}
resampled_features = {}
for feature_name, feature_data in features.items():
    resampled = np.zeros((total_frames, feature_data.shape[0]))
    for i in range(feature_data.shape[0]):
        resampled[:, i] = np.interp(np.linspace(0, len(feature_data[i])-1, total_frames), np.arange(len(feature_data[i])), feature_data[i])
    resampled_features[feature_name] = resampled

# Combine all features into a single DataFrame and save it
all_features = np.hstack([resampled_features[feature_name] for feature_name in ['Chroma', 'MFCC', 'MFCC_delta']])
feature_columns = ['Chroma_' + str(i) for i in range(chroma.shape[0])] + ['MFCC_' + str(i) for i in range(13)] + ['MFCC_delta_' + str(i) for i in range(13)]
features_df = pd.DataFrame(all_features, columns=feature_columns)
features_df.to_csv('/content/continuous_mfcc_features.csv', index=False)
print("Continuous MFCC, Delta MFCC, and Chroma features extracted and saved.")

# Load the DataFrame from the saved CSV file
features_df = pd.read_csv('/content/continuous_mfcc_features.csv')

# Function to normalize and save each feature column as a list in a text file
def save_and_normalize_features(df, prefix='/content/'):
    for column in df.columns:
        # Select and normalize the column
        data = df[column]
        normalized_data = (data - data.min()) / (data.max() - data.min())

        # Round the normalized data to two decimal places
        normalized_data = normalized_data.round(2)

        # Write formatted data to a file
        output_path = f"{prefix}{column}.txt"
        with open(output_path, 'w') as file:
            formatted_data = ',\n'.join([f"{i + 1}:({value})" for i, value in enumerate(normalized_data)])
            file.write(formatted_data)
        print(f"Data for {column} saved to: {output_path}")

# Normalize and save all columns as separate lists in text files
save_and_normalize_features(features_df)

Continuous MFCC, Delta MFCC, and Chroma features extracted and saved.
Data for Chroma_0 saved to: /content/Chroma_0.txt
Data for Chroma_1 saved to: /content/Chroma_1.txt
Data for Chroma_2 saved to: /content/Chroma_2.txt
Data for Chroma_3 saved to: /content/Chroma_3.txt
Data for Chroma_4 saved to: /content/Chroma_4.txt
Data for Chroma_5 saved to: /content/Chroma_5.txt
Data for Chroma_6 saved to: /content/Chroma_6.txt
Data for Chroma_7 saved to: /content/Chroma_7.txt
Data for Chroma_8 saved to: /content/Chroma_8.txt
Data for Chroma_9 saved to: /content/Chroma_9.txt
Data for Chroma_10 saved to: /content/Chroma_10.txt
Data for Chroma_11 saved to: /content/Chroma_11.txt
Data for MFCC_0 saved to: /content/MFCC_0.txt
Data for MFCC_1 saved to: /content/MFCC_1.txt
Data for MFCC_2 saved to: /content/MFCC_2.txt
Data for MFCC_3 saved to: /content/MFCC_3.txt
Data for MFCC_4 saved to: /content/MFCC_4.txt
Data for MFCC_5 saved to: /content/MFCC_5.txt
Data for MFCC_6 saved to: /content/MFCC_6.txt
Data

# Simple Beat Times Lists

In [ ]:
import os
import librosa
import numpy as np

#@markdown ### Path to your audio file
audio_path = "/content/Mechanical_Mirage.mp3" #@param {type:"string"}

# Load the audio file
y, sr = librosa.load(audio_path, sr=None)
# Extract tempo and beats
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beats, sr=sr)

# Round beat times to hundredth of a second
beat_times_rounded = [round(time, 2) for time in beat_times]
# Calculate frame numbers at 10 fps based on beat times
frame_numbers_10fps = [int(time * 10) for time in beat_times_rounded]
max_frame = max(frame_numbers_10fps)

# Initialize switch and inverse switch frames
current_state = 0
switch_frames = []
invert_switch_frames = []
for i in range(max_frame + 1):
    if i in frame_numbers_10fps:
        current_state = 1 - current_state  # Toggle between 0 and 1
    switch_frames.append(f"{i + 1}:({current_state})")
    invert_switch_frames.append(f"{i + 1}:({1 - current_state})")

# Save the switch effect frames
output_path_switch = '/content/beattimes_switch.txt'
with open(output_path_switch, 'w') as file:
    file.write(',\n'.join(switch_frames))
print(f"Switch effect frame list saved to: {output_path_switch}")

# Save the inverse switch effect frames
output_path_invert_switch = '/content/invert_beattimes_switch.txt'
with open(output_path_invert_switch, 'w') as file:
    file.write(',\n'.join(invert_switch_frames))
print(f"Inverse switch effect frame list saved to: {output_path_invert_switch}")

# Function to interpolate beat changes with smooth gradient
def interpolate_beat_changes(beat_times, fps=10):
    frame_numbers = [int(time * fps) for time in beat_times]
    beat_wave = np.zeros(max(frame_numbers) + 1)
    for i in range(len(frame_numbers) - 1):
        start_frame = frame_numbers[i]
        end_frame = frame_numbers[i + 1]
        beat_wave[start_frame:end_frame] = np.linspace(0, 1, end_frame - start_frame, endpoint=False)
    return beat_wave

# Generate and save the continuous beat wave
beat_wave = interpolate_beat_changes(beat_times_rounded)
output_path_wave = '/content/beatwaves.txt'
with open(output_path_wave, 'w') as file:
    formatted_data_wave = ',\n'.join([f"{i + 1}:({value:.2f})" for i, value in enumerate(beat_wave)])
    file.write(formatted_data_wave)
print(f"Continuous beat wave data saved to: {output_path_wave}")

# Generate and save the inverse continuous beat wave
inverse_beat_wave = 1 - beat_wave
output_path_inverse_wave = '/content/invert_beatwaves.txt'
with open(output_path_inverse_wave, 'w') as file:
    formatted_data_inverse_wave = ',\n'.join([f"{i + 1}:({value:.2f})" for i, value in enumerate(inverse_beat_wave)])
    file.write(formatted_data_inverse_wave)
print(f"Inverse continuous beat wave data saved to: {output_path_inverse_wave}")

Switch effect frame list saved to: /content/beattimes_switch.txt
Inverse switch effect frame list saved to: /content/invert_beattimes_switch.txt
Continuous beat wave data saved to: /content/beatwaves.txt
Inverse continuous beat wave data saved to: /content/invert_beatwaves.txt
